In [1]:
import os
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = 'calvinandpogs-ee148'

role = sagemaker.get_execution_role()
print(bucket, role)
print(os.getcwd())

calvinandpogs-ee148 arn:aws:iam::652516965730:role/service-role/AmazonSageMaker-ExecutionRole-20210513T011299
/home/ec2-user/SageMaker/atrw/models/yolov5


In [10]:
# TENSORFLOW CONTAINER (PY37)

from sagemaker.tensorflow import TensorFlow

# Store-true parameters not working

estimator = TensorFlow(entry_point='test.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 1920,
                            'task': 'test',
                            'weights': 'SM_CHANNEL_WEIGHT/best.pt',
                            'data': 'atrw.yaml'
                        }
)

In [12]:
estimator.fit({'weight': f's3://{bucket}/atrw/out/detection/yolov5/train/05-23-2021-10-53-02/runs/weights/',
               'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
               'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
               'test': f's3://{bucket}/atrw/detection/test/'})

2021-05-25 08:51:17 Starting - Starting the training job...
2021-05-25 08:51:42 Starting - Launching requested ML instancesProfilerReport-1621932674: InProgress
......
2021-05-25 08:52:42 Starting - Preparing the instances for training......
2021-05-25 08:53:42 Downloading - Downloading input data......
2021-05-25 08:54:42 Training - Downloading the training image..2021-05-25 08:54:59.350848: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-25 08:54:59.354260: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-25 08:54:59.459067: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-25 08:55:02,198 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-05-25 08:55:03,056 sagemaker-traini